# Entity Relationship Diagram (erd_v2)

Actualizamos el diagrama de entidad relación agregando algunos atributos a las entidades `Book` y `Author`.

![erd_02](https://raw.githubusercontent.com/maq-miguel-quinteros/00_apuntes/main/03_java-python/API%20RESTful%20Java%20and%20SpringBoot/wip/img/erd_02.png)

# New `Author` entity, repository & service

## Entity

Dentro del `package models` creamos una nueva clase llamada `Author` y la configuramos de forma similar a la clase `Book`.

In [ ]:
@Entity
public class Author {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;  

    public Author() {
    }

    public Author(String name, String lastName, LocalDate birthdate, LocalDate deathDate) {
        this.name = name;
        this.lastName = lastName;
        this.birthdate = birthdate;
        this.deathDate = deathDate;
    }

    // GETTERS Y SETTERS EXCEPTO EL setId
}

## Respostory & service

### Repository

Dentro de la `package repositories` creamos la interface llamada `AuthorRepository`.

In [ ]:
@RepositoryRestResource
public interface AuthorRepository extends JpaRepository <Author, Long> {
}

### DTO

Dentro de la `package dtos` creamos la clase llamada `AuthorDTO`

In [ ]:
public class AuthorDTO {

    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;

    public AuthorDTO() {
    }
    public AuthorDTO(Author author) {
        this.id = author.getId();
        this.name = author.getName();
        this.lastName = author.getLastName();
        this.birthdate = author.getBirthdate();
        this.deathDate = author.getDeathDate();
    }

    // SOLO GETTERS
}

### Service

Dentro de la `package services` creamos la interface llamada `AuthorService`.

In [ ]:
public interface AuthorService {

    // CREATE
    void create(Author author);

    // READ
    List<AuthorDTO> readAll();
}

#### Implement

Dentro del `package implement` creamos la clase llamada `AuthorServiceImplement`.

In [ ]:
@Service
public class AuthorServiceImplement implements AuthorService {

    @Autowired
    AuthorRepository authorRepository;
    
    @Override
    public void create(Author author) {
        authorRepository.save(author);
    }

    @Override
    public List<AuthorDTO> readAll() {
        return authorRepository.findAll().stream()
                .map(author -> new AuthorDTO(author))
                .collect(Collectors.toList());
    }
}

# Update `Book` entity 

Actualizamos la definición de la clase `Book` agregando el atributo `pagesNumber` de tipo `int`, sumando el mismo al constructor de la clase y creando su `getter` y `setter`.

In [ ]:
@Entity
public class Book {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    @Lob
    private String synopsis;
    private int pagesNumber;

    public Book() {}
    public Book(String isbn, String title, LocalDate date ,String synopsis, int pagesNumber) {
        this.isbn = isbn;
        this.title = title;
        this.date = date;
        this.synopsis = synopsis;
        this.pagesNumber = pagesNumber;
    }

    // GETTERS & SETTERS EXCEPTO setId

}

# Entity relations

Como podemos observar en el diagrama de entidad relación erd_v2, del comienzo del documento, y tomando en cuenta la lógica con la que desarrollamos el diagrama, que nos dice que un libro `Book` puede tener uno o mas autores `Author`, estableciendo una relación de uno a muchos (`1...N`) de libro a autor, y que un autor lo puede ser de uno o muchos libros, estableciendo el mismo tipo de relación (`1...N`) de autor a libro, nos vemos en la necesidad de crear una entidad intermedia, que denominamos `BookAuthor`, para establecer la relación entre libros y autores, y normalizar estas tablas en la base de datos.

## `BookAuthor` entity

Dentro del `package models` creamos una nueva clase llamada `BookAuthor` y la configuramos de la siguiente manera.

* `@ManyToOne`: mediante la `annotation @ManyToOne` indicamos que la entidad generada a partir de la clase `BookAuthor` va a tener una relación de muchos a uno (`N...1`) con el tipo del atributo que está debajo de la `annotation`, que en ejemplo son `book` y `author`, quiere decir que, mediante esta `annotation`, estamos configurando la relación de muchos a uno entre la entidad `BookAuthor` con respecto a la entidad `Book`, por un lado, y la entidad `Author` por el otro.

* `fetch = FetchType.EAGER`: mediante el parámetro `fetch`, que pasamos a la `annotation`, indicamos la forma en que va a ser cargada la entidad. Al parámetro le asignamos el valor `FetchType.EAGER` que indica que la relación debe ser cargada al momento de cargar la entidad.
* `@JoinColumn`(name = "book_id"): mediante la `annotation @JoinColumn` indicamos que el atributo que está debajo va a requerir de un `Join` para poder acceder al mismo, es decir que el atributo viene dado por una relación con otra entidad, en este caso por la entidad `Book`, y la columna en la base de datos correspondiente a este atributo se configura a partir de esa relación.
* `name = "book_id"` y `name = "book_id"`: mediante el parámetro `name` que pasamos a la `annotation` indicamos el nombre de la columna en la base de datos para los atributos de tipo `Book` y tipo `Author`, que en el ejemplo va a ser `"book_id"` y `author_id`.
* No generamos el constructor con parámetros para la clase por que el único atributo propio de la misma, `id`, se genera de forma automática por la `annotation @GeneratedValue`. Los atributos `book` y `author` pertenecen a la relación con las entidades de `Book` y `Author` por lo que no se le asignan atributos desde un constructor. Si la clase tuviese atributos propios además de `id` correspondería generar el constructor.

In [ ]:
@Entity
public class BookAuthor {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;

    @ManyToOne(fetch = FetchType.EAGER)
    @JoinColumn(name = "book_id")
    private Book book;

    @ManyToOne(fetch = FetchType.EAGER)
    @JoinColumn(name = "author_id")
    private Author author;

    public BookAuthor() {
    }
    // GETTERS & SETTERS EXCEPTO EL setId
}

## `BookAuthor` repository & service

### Repository

Dentro de la `package repositories` creamos la interface llamada `BookAuthorRepository`.

In [ ]:
@RepositoryRestResource
public interface BookAuthorRepository extends JpaRepository <BookAuthor, Long> {
}

### DTO

Dentro de la `package dtos` creamos la clase llamada `BookAuthorDTO` y la configuramos de la siguiente manera.

* Los atributos del dto no tienen que ser, necesariamente, los atributos de la clase a la que refiere el mismo. La definición de estos atributos va a depender de cuales sean los datos que necesitemos pasar al frontend, esto quiere decir que usamos el dto como su nombre indica, data transfer object, para pasar mediante un objeto los datos que queremos pasar.

* La clase del dto tiene atributos propios del dto como `author_name`, `author_lastName`, `author_birthdate` y `author_deathDate` que van a ser asignados en base al atributo de tipo `Author` del objeto `BookAuthor` que pasamos a la constructor del dto.
* `bookAuthor.getAuthor().getName()`: mediante este método `getAuthor()`, del objeto `bookAuthor`, traemos del objeto de tipo `Author` que este tiene, y mediante el método `getName()` del objeto de tipo `Author` traemos el atributo `name` de este.

In [ ]:
public class BookAuthorDTO {

    private Long id;
    private String author_name;
    private String author_lastName;
    private LocalDate author_birthdate;
    private LocalDate author_deathDate;

    public BookAuthorDTO() {
    }

    public BookAuthorDTO(BookAuthor bookAuthor) {
        this.id = bookAuthor.getId();
        this.author_name = bookAuthor.getAuthor().getName();
        this.author_lastName = bookAuthor.getAuthor().getLastName();
        this.author_birthdate = bookAuthor.getAuthor().getBirthdate();
        this.author_deathDate = bookAuthor.getAuthor().getDeathDate();
    }

    // SOLO GETTERS
}

### Service

Dentro de la `package services` creamos la interface llamada `BookAuthorService`.

In [ ]:
public interface BookAuthorService {

    // CREATE
    void create(BookAuthor bookAuthor);

}

#### Implement

Dentro del `package implement` creamos la clase llamada `BookAuthorServiceImplement`.

In [ ]:
@Service
public class BookAuthorServiceImplement implements BookAuthorService {

    @Autowired
    BookAuthorRepository bookAuthorRepository;

    @Override
    public void create(BookAuthor bookAuthor) {
        bookAuthorRepository.save(bookAuthor);
    }
}

## Update `Book` entity

Actualizamos la definición de la clase `Book` de la siguiente manera.

* `@OneToMany`: mediante la `annotation @OneToMany` indicamos que la entidad generada a partir de la clase `Book` va a tener una relación de uno a muchos (`1...N`) con el tipo del atributo que está debajo de la `annotation`, que en ejemplo es `BookAuthor`, quiere decir que, mediante esta `annotation`, estamos configurando la relación de uno a muchos entre la entidad `Book` con respecto a la entidad `BookAuthor`.

* `mappedBy = "book"`: mediante el parámetro `mappedBy`, que pasamos a la `annotation`, indicamos el nombre de la columna que, en la base de datos, van a tener las entradas correspondientes al listado de autores del libro, listado que puede tener uno o muchos autores.
* `Set<BookAuthor> authors`: al indicar que el tipo de datos del atributo `author` es `Set<BookAuthor>` indicamos que el mismo va a ser un `Set`, un tipo de datos que representa un listado, de objetos de tipo `BookAuthor`.
* `new HashSet<>()`: mediante la interface `HashSet` generamos los objetos para el listado de tipo `Set`.
* El atributo `authors` pertenece a la relación con la entidad `BookAuthor` por lo que no se le asignan valores desde un constructor.
* `addBookAuthor(BookAuthor bookAuthor)`: generamos el método `addBookAuthor`, que recibe como parámetro un objeto de tipo `BookAuthor`, mediante el cual vamos a sumar autores al listado de estos que se corresponde con el atributo `authors`.
* `bookAuthor.setBook(this)`: llamamos al método `setBook`, del objeto `BookAuthor` que pasamos por parámetro, y mediante el parámetro `this` indicamos que se asigne el valor del libro actual, con todos sus atributos, al atributo de tipo `book` del objeto de tipo `BookAuthor`.
* `authors.add(bookAuthor)`: el método `add`, que hereda el atributo `authors` por ser de tipo `Set`, agrega un nuevo objeto de tipo `BookAuthor` al listado de autores del libro.

In [ ]:
@Entity
public class Book {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    @Lob
    private String synopsis;
    private int pagesNumber;
    @OneToMany(mappedBy = "book", fetch = FetchType.EAGER)
    private Set<BookAuthor> authors = new HashSet<>();

    public Book() {}
    public Book(String isbn, String title, LocalDate date ,String synopsis, int pagesNumber) {
        this.isbn = isbn;
        this.title = title;
        this.date = date;
        this.synopsis = synopsis;
        this.pagesNumber = pagesNumber;
    }

    public void addBookAuthor(BookAuthor bookAuthor){
        bookAuthor.setBook(this);
        authors.add(bookAuthor);
    }

    // GETTERS & SETTERS EXCEPTO setId
}

## Update `Author` entity

Actualizamos la definición de la clase `Author` de la misma forma que la clase `Book`, creando un atributo llamado `books` definido como un listado `Set` de objetos `BookAuthor`.

In [ ]:
@Entity
public class Author {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;
    @OneToMany(mappedBy = "author", fetch = FetchType.EAGER)
    private Set<BookAuthor> books = new HashSet<>();

    public Author() { }
    public Author(String name, String lastName, LocalDate birthdate, LocalDate deathDate) {
        this.name = name;
        this.lastName = lastName;
        this.birthdate = birthdate;
        this.deathDate = deathDate;
    }

    public void addBookAuthor(BookAuthor bookAuthor){
        bookAuthor.setAuthor(this);
        books.add(bookAuthor);
    }

    // 
}

## Update `BookDTO` class

In [ ]:
public class BookDTO {

    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    private String synopsis;
    private int pagesNumber;
    private Set<BookAuthorDTO> authors;

    public BookDTO() {
    }
    public BookDTO(Book book) {
        this.id = book.getId();
        this.isbn = book.getIsbn();
        this.title = book.getTitle();
        this.date = book.getDate();
        this.synopsis = book.getSynopsis();
        this.pagesNumber = book.getPagesNumber();

        this.authors = book.getAuthors().stream()
                .map(bookAuthor -> new BookAuthorDTO(bookAuthor))
                .collect(Collectors.toSet());
    }

    // SOLO LOS GETTERS
}